In [9]:
import math

from qiskit import QuantumCircuit
import numpy as np

pi = np.pi

##
##circuit.cp(theta,i,j). ith control and j control bit
##circuit.swap(i,j)
for i in range(10, 0, -1):
    print(i)

10
9
8
7
6
5
4
3
2
1


In [101]:
# n = 3
n = 4
circuit = QuantumCircuit(n, 0)


def apply_qft(n: int, circuit: QuantumCircuit) -> QuantumCircuit:
    for i in range(circuit.num_qubits - 1, -1, -1):
        circuit.barrier()
        circuit.h(i)
        for k in range(i, 0, -1):
            circuit.cp(pi * (np.exp2(k - i - 1)), i, k - 1);

    circuit.barrier()
    for i in range(circuit.num_qubits // 2):
        circuit.swap(i, n - i - 1)
    return circuit


apply_qft(n, circuit)
circuit.draw()

░                                  ░                         ░      »
q_0: ─░─────────────────────────■────────░────────────────■────────░──────»
      ░                         │        ░                │        ░ ┌───┐»
q_1: ─░────────────────■────────┼────────░───────■────────┼────────░─┤ H ├»
      ░                │        │        ░ ┌───┐ │P(π/2)  │P(π/4)  ░ └───┘»
q_2: ─░───────■────────┼────────┼────────░─┤ H ├─■────────■────────░──────»
      ░ ┌───┐ │P(π/2)  │P(π/4)  │P(π/8)  ░ └───┘                   ░      »
q_3: ─░─┤ H ├─■────────■────────■────────░─────────────────────────░──────»
      ░ └───┘                            ░                         ░      »
«               ░ ┌───┐ ░       
«q_0: ─■────────░─┤ H ├─░──X────
«      │P(π/2)  ░ └───┘ ░  │    
«q_1: ─■────────░───────░──┼──X─
«               ░       ░  │  │ 
«q_2: ──────────░───────░──┼──X─
«               ░       ░  │    
«q_3: ──────────░───────░──X────
«               ░       ░

In [128]:
import math

n = 4
circuit = QuantumCircuit(n, 0)


def apply_iqft(n: int, circuit: QuantumCircuit) -> QuantumCircuit:
    for i in range((circuit.num_qubits // 2)):
        circuit.swap(i, n - i - 1)
    circuit.barrier()

    for j in range(circuit.num_qubits):
        for k in range(j):
            circuit.cp(-pi * (np.exp2(k - j)), j, k);
        circuit.h(j)
        circuit.barrier()


apply_iqft(n, circuit)
print(circuit)

            ░ ┌───┐ ░                 ░                           ░           »
q_0: ─X─────░─┤ H ├─░──■──────────────░──■────────────────────────░──■────────»
      │     ░ └───┘ ░  │P(-π/2) ┌───┐ ░  │                        ░  │        »
q_1: ─┼──X──░───────░──■────────┤ H ├─░──┼─────────■──────────────░──┼────────»
      │  │  ░       ░           └───┘ ░  │P(-π/4)  │P(-π/2) ┌───┐ ░  │        »
q_2: ─┼──X──░───────░─────────────────░──■─────────■────────┤ H ├─░──┼────────»
      │     ░       ░                 ░                     └───┘ ░  │P(-π/8) »
q_3: ─X─────░───────░─────────────────░───────────────────────────░──■────────»
            ░       ░                 ░                           ░           »
«                               ░ 
«q_0: ──────────────────────────░─
«                               ░ 
«q_1: ─■────────────────────────░─
«      │                        ░ 
«q_2: ─┼─────────■──────────────░─
«      │P(-π/4)  │P(-π/2) ┌───┐ ░ 
«q_3: ─■─────────■────────┤ H ├─░─


In [130]:

def apply_iqft_gen(qubits: list, circuit):
    n = len(qubits)
    for i in range((circuit.num_qubits // 2)):
        circuit.swap(qubits[i], qubits[n - i - 1])
    circuit.barrier()

    for j in range(circuit.num_qubits):
        for k in range(j):
            circuit.cp(-pi * (np.exp2(k - j)), qubits[j], qubits[k]);
        circuit.h(qubits[j])
        circuit.barrier()


def apply_qft_gen(qubits: list, circuit: QuantumCircuit) -> QuantumCircuit:
    n = len(qubits)
    for i in range(circuit.num_qubits - 1, -1, -1):
        circuit.barrier()
        circuit.h(qubits[i])
        for k in range(i, 0, -1):
            circuit.cp(pi * (np.exp2(k - i - 1)), qubits[i], qubits[k - 1]);

    circuit.barrier()
    for i in range(circuit.num_qubits // 2):
        circuit.swap(qubits[i], qubits[n - i - 1])
    return circuit


n = 4
c = QuantumCircuit(n, 0)
apply_iqft_gen([0, 1, 2, 3], c)
print(c)


            ░ ┌───┐ ░                 ░                           ░           »
q_0: ─X─────░─┤ H ├─░──■──────────────░──■────────────────────────░──■────────»
      │     ░ └───┘ ░  │P(-π/2) ┌───┐ ░  │                        ░  │        »
q_1: ─┼──X──░───────░──■────────┤ H ├─░──┼─────────■──────────────░──┼────────»
      │  │  ░       ░           └───┘ ░  │P(-π/4)  │P(-π/2) ┌───┐ ░  │        »
q_2: ─┼──X──░───────░─────────────────░──■─────────■────────┤ H ├─░──┼────────»
      │     ░       ░                 ░                     └───┘ ░  │P(-π/8) »
q_3: ─X─────░───────░─────────────────░───────────────────────────░──■────────»
            ░       ░                 ░                           ░           »
«                               ░ 
«q_0: ──────────────────────────░─
«                               ░ 
«q_1: ─■────────────────────────░─
«      │                        ░ 
«q_2: ─┼─────────■──────────────░─
«      │P(-π/4)  │P(-π/2) ┌───┐ ░ 
«q_3: ─■─────────■────────┤ H ├─░─
